In [10]:
import pandas as pd


In [15]:
df=pd.read_csv("/home/amin/equiformer/data/qm9-precise3d/raw/gdb9.sdf.csv")
df["homo"].mean()

-0.23997669940620683

In [7]:
def get_tasks(df):
    # Get column names that start with 'pred_', 'actual_', or 'diff_'
    pred_cols = [col for col in df.columns if col.startswith('pred_')]
    tasks = [col.replace('pred_','') for col in pred_cols]
    
    return tasks

In [9]:
structure_list=["precise3d","optimized3d","rdkit3d","rdkit2d"]
dataset="Esol"
for structure in structure_list:
    df=pd.read_csv(f"/home/amin/equiformer/scripts/output/{dataset}-{structure}/test_results.csv")
    tasks=get_tasks(df)
    dataset_error=0
    for task in tasks:
        task_error=df[f'diff_{task}'].abs().mean()
        dataset_error+=task_error
        print(f"structure {structure}-{task}:{task_error}")
    print(f"structure {structure}:{dataset_error/len(tasks)}")

structure precise3d-measured log solubility in mols per litre:0.6990828288679246
structure precise3d:0.6990828288679246
structure optimized3d-measured log solubility in mols per litre:0.7139583149469027
structure optimized3d:0.7139583149469027
structure rdkit3d-measured log solubility in mols per litre:0.6585405857247787
structure rdkit3d:0.6585405857247787
structure rdkit2d-measured log solubility in mols per litre:0.6637784696371681
structure rdkit2d:0.6637784696371681


86.47916823550662